## Upper bound on multiple rank-one updates

Zhu provides good bounds on the rank-one update eigenvalues with no diagonalization at all (and calculation of each eigenvalue is completely parallel). However, the experiment here, tracking only the top 2 eigenvalues, was not a success. If the upper bound on the third eigenvalue is not tight, then the upper bound on the second eigenvalue will not be tight on the next rank-one update (and so on).

Also, the angle could change by 1/i (1/3 here) radians or so, if the new variable has a dramatically high loading in a principal component other than the dominant eigenvalue. In those cases, the dominant eigenvalue will fail to grow, so that particular path will probably not lead to the upper bound. However, we should max our candidate eigenvalue/eigenvector upper bounds/uncertainty bound independently. Now, in the cases where the dominant eigenvalue grew the most so the angle of uncertainty is small, then it is tainted with some cases where we explored and grew the other principal components but not the dominant one.

Finally, on the next iteration, all inputs to Zhu will be some bounded unknowns. We haven't solved for critical points of this multivariate function yet. We solved for the gradient w.r.t. gap, and found the saddle point elegantly. We shouldn't proceed without solving for all gradients equal to 0, with a good closed-form. This would be a good explanation for the difficulty of bounding a general update to a matrix, if we cannot get a closed-form for all extrema in the original bounds to be tested.

In [3]:
import numpy as np
np.set_printoptions(linewidth = 150, precision = 4, suppress = True)

In [4]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(float)
data

array([[ 1.    , -0.0461,  0.2312, ...,  0.2704,  0.4664,  0.3672],
       [-0.0461,  1.    , -0.0671, ..., -0.0515, -0.0944, -0.0349],
       [ 0.2312, -0.0671,  1.    , ...,  0.147 ,  0.2608,  0.6313],
       ...,
       [ 0.2704, -0.0515,  0.147 , ...,  1.    ,  0.2066,  0.1538],
       [ 0.4664, -0.0944,  0.2608, ...,  0.2066,  1.    ,  0.3486],
       [ 0.3672, -0.0349,  0.6313, ...,  0.1538,  0.3486,  1.    ]])

In [5]:
nodes = np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
nodes = nodes[1:].astype(float)
bounds = np.loadtxt('../NC-K7-Trace-Bounds.csv', delimiter=',', dtype=str)
bounds = bounds[1:].astype(float)

In [7]:
np.argmax((nodes == 1).sum(axis=0) == 3)

42

In [8]:
bounds[:, 42]

array([4.1355, 5.8707, 7.    ])

In [9]:
(nodes[:, 42] != 0).sum()

82

In [179]:
selected = np.where(nodes[:, 42] == 1)[0]
selected

array([ 3, 83, 85])

In [68]:
S, V = np.linalg.eigh(data[selected, :][:, selected])
S

array([0.0114, 0.9687, 2.0199])

In [82]:
def zd(arr):
    arr = arr.copy()
    np.fill_diagonal(arr, 0)
    return arr

In [195]:
data_clr = data.copy()
data_clr[selected, selected[0]] = 0
data_clr[selected, selected[1]] = 0
data_clr[selected, selected[2]] = 0
np.fill_diagonal(data_clr, 0)
np.divmod(np.argmin(abs(data_clr[selected, 4:83]).max(axis=0)), 101)

(0, 31)

In [196]:
data[3, 35]

0.027496576592458615

In [183]:
next_ind = 83

In [197]:
next_ind = 35

In [229]:
# Say that we were already tracking the third smallest eigenvalue by a cheap method (Gershgorin upper bound).
np.linalg.eigh(data[selected[:2], :][:, selected[:2]])[0][0]

0.8732683226024542

In [198]:
np.linalg.eigh(data[list(selected) + [next_ind], :][:, list(selected) + [next_ind]])[0]

array([0.0113, 0.9471, 1.0207, 2.0209])

In [199]:
import scipy.linalg
w = data[selected, next_ind:next_ind + 1]
w = np.r_[w, [[0.]]]
# WRONG!
# np.linalg.eigh(
#     scipy.linalg.block_diag(data[selected, :][:, selected], np.zeros((1, 1)))
#     + w.dot(w.T))[0]

w = scipy.linalg.block_diag(V.T, np.ones((1, 1))).dot(w) / np.sqrt(list(S) + [1])[:, None]
w[-1, 0] = np.sqrt(1. - np.linalg.norm(w[:-1, :]) ** 2)
np.linalg.eigh(
    np.diag(list(S) + [0.])
    + w.dot(w.T)
)[0]

array([0.0113, 0.9471, 1.0207, 2.0209])

In [200]:
w

array([[-0.1084],
       [-0.0341],
       [ 0.0225],
       [ 0.9933]])

In [227]:
def l1(w1, w2, gap):
    return (w1 ** 2 + w2 ** 2 - gap + np.sqrt((gap + w1 ** 2 + w2 ** 2) ** 2 - 4 * gap * w2 ** 2)) / 2.
def u1(w1, gap):
    return (1. - gap + np.sqrt((gap + 1) ** 2 - 4 * gap * (1. - w1 ** 2))) / 2.
def l2(w1, w2, gap):
    return (gap + w1 ** 2 + w2 ** 2 - np.sqrt((gap + w1 ** 2 + w2 ** 2) ** 2 - 4 * gap * w2 ** 2)) / 2.
def u2(w1, w2, eig_2):
    # eig_3 is unknown. We could try to track eig_3 using Gershgorin
    gap = np.linspace(eig_2 - 0.8733, eig_2, 1000)
    # gap = np.linspace(0, eig_2, 1000)
    values = (1. - w1 ** 2 - gap + np.sqrt((gap + 1 - w1 ** 2) ** 2 - 4 * gap * (1 - w1 ** 2 - w2 ** 2))) / 2.
    return values.max()

In [202]:
S[2] + l1(w[2, 0], w[1, 0], S[2] - S[1]), S[2] + u1(w[2, 0], S[2] - S[1])

(2.0203882918334886, 2.0287715881043322)

In [230]:
(S[1] + l2(w[2, 0], w[1, 0], S[2] - S[1]),
 S[1] + u2(w[2, 0], w[1, 0], S[1]))

(0.9698660362330068, 1.8729143835255053)

In [231]:
upper = (u1(V.T.dot(data[selected, next_ind:next_ind+1])[-1, 0] / np.sqrt(S[-1]), S[2] - S[1]))
second = (S[1] - (S[2] + l1(w[2, 0], w[1, 0], S[2] - S[1])))
upper, second
z = 1. / np.r_[
    S[0] - (S[2] + l1(w[2, 0], w[1, 0], S[2] - S[1])),
    second,
    upper,
    -(S[2] + l1(w[2, 0], w[1, 0], S[2] - S[1])),
][:, None] * w
z /= np.linalg.norm(z)
z[2, 0], np.arccos(np.abs(z[2, 0]))

(0.9814177270273432, 0.1930808639341894)

In [232]:
abs(w * w[-1, 0]).sum()

1.1504706369062427